<a href="https://colab.research.google.com/github/jocelyndunstan/MA5406/blob/main/Clase_3_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Definir directorio e importar paquetes

In [4]:
pip install Unidecode

     |████████████████████████████████| 245kB 5.5MB/s 


In [25]:
import os
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import pandas as pd
import numpy as np
import glob
import codecs
import re
from collections import Counter 
encoding = 'unicode_escape' 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Uso la RAE como diccionario

In [7]:
!wget https://raw.githubusercontent.com/jocelyndunstan/MA5406/main/dict.txt

--2021-04-12 19:33:03--  https://raw.githubusercontent.com/jocelyndunstan/MA5406/main/dict.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1170904 (1.1M) [text/plain]
Saving to: ‘dict.txt’

dict.txt            100%[===================>]   1.12M  --.-KB/s    in 0.08s   

2021-04-12 19:33:04 (14.6 MB/s) - ‘dict.txt’ saved [1170904/1170904]



In [8]:
corpus_raw = u""

with codecs.open("dict.txt", 'r','utf-8') as book_file:
        corpus_raw += book_file.read()
    
print("Corpus is now {0} characters long".format(len(corpus_raw)))


Corpus is now 1147060 characters long


In [9]:
corpus_raw

'ABS\r\nADN\r\nADSL\r\nAM\r\nARN\r\nAVE\r\nAbraham\r\nAdviento\r\nAgramante\r\nAirón\r\nAlcalá\r\nAlcorán\r\nAlcántara\r\nAlejandría\r\nAlicante\r\nAllen\r\nAltísimo\r\nAlzheimer\r\nAmérica\r\nAméricas\r\nAmón\r\nAnchuelo\r\nAndrés\r\nAntillas\r\nAntonio\r\nAntón\r\nAquiles\r\nAragón\r\nArmenia\r\nArquímedes\r\nBabia\r\nBamba\r\nBarcelona\r\nBatavia\r\nBatuecas\r\nBayona\r\nBañón\r\nBierzo\r\nBlas\r\nBohemia\r\nBona\r\nBrasil\r\nBright\r\nBriján\r\nBruselas\r\nBuenosAires\r\nBurgos\r\nBárbara\r\nCD\r\nCDROM\r\nCabo\r\nCachano\r\nCalabar\r\nCalatrava\r\nCalaínos\r\nCalañas\r\nCalleja\r\nCanadá\r\nCancelaría\r\nCaplán\r\nCaravaca\r\nCardona\r\nCarracuca\r\nCasio\r\nCeilán\r\nCelsius\r\nChagas\r\nChiapa\r\nChile\r\nCobos\r\nCompostela\r\nCongreve\r\nConstantinopla\r\nCoria\r\nCorinto\r\nCorpus\r\nCorán\r\nCribas\r\nCuasimodo\r\nCuenca\r\nCástoryPólux\r\nDDT\r\nDIN\r\nDNA\r\nDNI\r\nDVD\r\nDamocles\r\nDavid\r\nDeovolente\r\nDomingo\r\nDown\r\nEdipo\r\nEgeria\r\nEgipto\r\nElEscorial\r\nElect

## Código que reemplaza palabras si es necesario

Modificado de http://norvig.com/spell-correct.html

In [12]:
def words(text): return re.findall(r'\w+', text.lower())

# LA FUNCION COUNTER GUARDA PALABRAS CON SU PROBABILIDAD DE APARICION EN EL TEXTO DE ENTRENAMIENTO
WORDS = Counter(words(corpus_raw))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

# PRIMERO CHEQUEA SI LA PALABRA ESTA EN EL DICCIONARIO. SI ESTA, ENTONCES DEVUELVE LA MISMA PALABRA"
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

# ELIGE LA PALABRA MAS PROBABLE QUE ESTA A UNA OPERACION DE LA PALABRA DE ENTRADA
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnñopqrstuvwxyzáéíóú'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# ELIGE LA PALABRA MAS PROBABLE QUE ESTA A DOS OPERACION DE LA PALABRA DE ENTRADA
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Algunos chequeos

In [13]:
correction("rcorporacion")

'corporación'

In [14]:
correction("rencorporacion")

'reincorporación'

In [15]:
correction("encorporacion")

'incorporación'

In [16]:
correction("pallaso")

'pallas'

In [17]:
correction("lord")

'lord'

Nótese que la palabra "lord" está en la RAE!  

In [19]:
"lord" in WORDS

True

## Leo un dictamen para testear el algortimo

In [21]:
!wget https://raw.githubusercontent.com/jocelyndunstan/MA5406/main/dictamen.txt

--2021-04-12 19:35:57--  https://raw.githubusercontent.com/jocelyndunstan/MA5406/main/dictamen.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2600 (2.5K) [text/plain]
Saving to: ‘dictamen.txt’

dictamen.txt        100%[===================>]   2.54K  --.-KB/s    in 0s      

2021-04-12 19:35:57 (48.9 MB/s) - ‘dictamen.txt’ saved [2600/2600]



In [22]:
f = open('dictamen.txt', encoding="ISO-8859-1")
raw = f.read()
f.close()

In [23]:
raw

'SUPCJ~Tl!~liEKfl,\\ \nVAL01r;$ YSEGl.l:ltU~ \n1\nOFORD.: N"6887 \nOficio N"2162, de 26/01/2011, de la !\nAn ecedentes .: Superintendencia de Pensiones. \ni \nInstruye reincorporación de beneficiaria ¿pensión\nMateria.: de sobrevivencia. SGD.: N"2011030043591 Santiago, 1Ode Marzo de 2011 \nDe: SuperintendenJia de Valores y Seguros \nA : ~~;~~~~~~~ .\nSEGUROS DE VIDA S. EL BOSQUE NORTE 0125 LAS CONDES fcomuna: SANTIAGO -Ciudad: SANTIAGO -Reg. Me rrolitana \n----------IT__________ --------1 \ni \nSe ha recibido el oficio del antecedente, el cual se ~djuntu, en que la Superintendencia de Pensiones señal~ que la AFP I-labitat y esa aseg~radora incurrieron en un errbr ~l eliminar de la póliza de renta vitalicia previsiona.lld,el Sr.\x95.:.-. \xadl f·--~ -\n.a lalbefieficiaria de pensión de sobrevivencia, Sra . ,__ ~· _.. :; ·..i· _. . ._ \n. i . y (iue en consecuencia ésta debb ser restituida en calidad de beneficiaria como Madr~ de Hijo de \nFiliación no fyfatri onial debido a que su matri

### Separo el texto en palabras, las paso a minusculas, y reemplazo todos los caracteres no-alfabeticos dentro de las palabras

In [26]:
v1 = nltk.word_tokenize(raw)
v2 = [w.lower() for w in v1]
v22 = []
for word in v2:
    word = re.sub('[^A-Za-z]', '', word)
    if len(word) == 0:
        pass
    else:
        v22.append(word)

In [ ]:
v22

['supcjtl',
 'liekfl',
 'valr',
 'ysegll',
 'ltu',
 'oford',
 'n',
 'oficio',
 'n',
 'de',
 'de',
 'la',
 'an',
 'ecedentes',
 'superintendencia',
 'de',
 'pensiones',
 'i',
 'instruye',
 'reincorporacin',
 'de',
 'beneficiaria',
 'pensin',
 'materia',
 'de',
 'sobrevivencia',
 'sgd',
 'n',
 'santiago',
 'ode',
 'marzo',
 'de',
 'de',
 'superintendenjia',
 'de',
 'valores',
 'y',
 'seguros',
 'a',
 'seguros',
 'de',
 'vida',
 's',
 'el',
 'bosque',
 'norte',
 'las',
 'condes',
 'fcomuna',
 'santiago',
 'ciudad',
 'santiago',
 'reg',
 'me',
 'rrolitana',
 'it',
 'i',
 'se',
 'ha',
 'recibido',
 'el',
 'oficio',
 'del',
 'antecedente',
 'el',
 'cual',
 'se',
 'djuntu',
 'en',
 'que',
 'la',
 'superintendencia',
 'de',
 'pensiones',
 'seal',
 'que',
 'la',
 'afp',
 'ilabitat',
 'y',
 'esa',
 'asegradora',
 'incurrieron',
 'en',
 'un',
 'errbr',
 'l',
 'eliminar',
 'de',
 'la',
 'pliza',
 'de',
 'renta',
 'vitalicia',
 'previsionalld',
 'el',
 'sr',
 'l',
 'f',
 'a',
 'lalbefieficiaria',
 

### Aplico el corrector

In [27]:
v3=v22.copy()
for i in range(len(v22)): 
    v3[i]= correction(v22[i])

### Muestro las palabras modificadas por el corrector

In [28]:
#comparacion = {'original':v22,'sugerencias':v3}
#df = pd.DataFrame(comparacion)
diferencias = pd.DataFrame([[i,j] for i, j in zip(v22, v3) if i != j])
diferencias.columns = ['Original', 'Sugerencia']
diferencias

,Original,Sugerencia
0,valr,valí
1,ltu,tu
2,oford,fora
3,ecedentes,excedente
4,pensiones,pensionar
...,...,...
99,tdrdar,tardar
100,pordend,orden
101,ilf,ilo
102,sgdi,sed


## Reescribir el dictamen usando el corrector

In [29]:
f = open('dictamen.txt', encoding="ISO-8859-1")
raw = f.read()
v1 = nltk.word_tokenize(raw)
v2 = [w.lower() for w in v1] 
v22 = []
for word in v2:
    word = re.sub('[^A-Za-z]', '', word)
    if len(word) == 0:
        pass
    else:
        v22.append(word)

In [30]:
corrected = []
for i in range(len(v22)): 
    corrected.append(correction(v22[i]))

In [ ]:
corrected

['supcjtl',
 'liekfl',
 'valí',
 'ysegll',
 'tu',
 'afore',
 'n',
 'oficio',
 'n',
 'de',
 'de',
 'la',
 'an',
 'cedente',
 'superintendencia',
 'de',
 'pensionar',
 'i',
 'instruir',
 'reincorporación',
 'de',
 'beneficiaria',
 'pensil',
 'materia',
 'de',
 'sobrevivencia',
 'sed',
 'n',
 'santiago',
 'de',
 'marzo',
 'de',
 'de',
 'superintendencia',
 'de',
 'valor',
 'y',
 'seguro',
 'a',
 'seguro',
 'de',
 'vida',
 's',
 'el',
 'bosque',
 'porte',
 'las',
 'conde',
 'comuna',
 'santiago',
 'ciudad',
 'santiago',
 'red',
 'me',
 'rrolitana',
 't',
 'i',
 'se',
 'ha',
 'recibido',
 'el',
 'oficio',
 'del',
 'antecedente',
 'el',
 'cual',
 'se',
 'junta',
 'en',
 'que',
 'la',
 'superintendencia',
 'de',
 'pensionar',
 'real',
 'que',
 'la',
 'ave',
 'habitat',
 'y',
 'esa',
 'aserradora',
 'incurrieron',
 'en',
 'un',
 'error',
 'l',
 'eliminar',
 'de',
 'la',
 'liza',
 'de',
 'renta',
 'vitalicia',
 'previsionalld',
 'el',
 'er',
 'l',
 'f',
 'a',
 'lalbefieficiaria',
 'de',
 'pensi

In [ ]:
#with open('corregida.txt', mode='wt') as myfile:
#    myfile.write(' '.join(corrected))